<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_Yoga_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the LICENSE file in the root directory of this source tree.

#Locating specific Yogs in horoscope


*   Budhaditya_Yog
*   Gajakesari_Yog
*   Raj_Yog



For those who may not have access to a live MongoDB instance, we have set up a mechanism where the JSON file that would have been loaded into a hosted MongoDB instance, will be loaded into a locally (that is in the VM) installed MongoDB instance and the queries can be run in a manner that is identical to a live, hosted MongoDB instance

#SetUp Environment

##Install Software

In [1]:
!pip -qq install python-docx                #https://python-docx.readthedocs.io/en/latest/
!apt install mongodb > /dev/null            # install local MongoDB server           
!service mongodb start                      # start Local MongoDB server instance


     |████████████████████████████████| 5.6 MB 7.4 MB/s 


 * Starting database mongodb
   ...done.


##Install, import prerequisites

In [2]:
#Utility functions 
from pymongo import MongoClient
import pandas as pd
import dateutil
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from datetime import timedelta
import pytz

#import matplotlib.pyplot as plt
#import math


In [3]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py 
# ---------------------------

!wget -O Saraswati.png -q https://github.com/prithwis/parashar21/raw/main/images/Saraswati02.png                 
!wget -O p21logo.png -q https://github.com/prithwis/parashar21/raw/main/images/p21logo-002-1.png                  

In [4]:
#!cat p21utils.py
import p21
import p21utils
import p21utilsR
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21utils)
importlib.reload(p21utilsR)

imported p21
p21utils imported
p21utilsR imported
imported p21
p21utils imported
p21utilsR imported


<module 'p21utilsR' from '/content/p21utilsR.py'>

##Define MongoDB database in local VM

In [5]:
mongoURI = "mongodb://localhost:27017/"                               # for remote server
client = MongoClient(mongoURI)

db = client.parashar21Local
#pd.DataFrame(list(db.list_collections()))

##Download Horoscope data in JSON and load into MongoDB

In [6]:
# Retrieve JSON file from Github and load in local Mongo
#!wget -O peopleData.json -q https://github.com/prithwis/parashar21/raw/main/data/peopleDataTest5.json

#!gdown -O peopleData.json https://drive.google.com/uc?id=1_VBHE7Hyue8t3hdBdLU6lgpnaWhhRVd4  # 39662 documents OLD file
!gdown  -O peopleData.json https://drive.google.com/uc?id=1z4JFd-zCaS71dICKIYx46kQGbXPAHq5C  # 39662 documents
!mongoimport --db parashar21Local --collection khona21Local --drop --file peopleData.json

Downloading...
From: https://drive.google.com/uc?id=1z4JFd-zCaS71dICKIYx46kQGbXPAHq5C
To: /content/peopleData.json
100% 134M/134M [00:01<00:00, 85.3MB/s]
2022-06-16T11:57:39.376+0000	connected to: localhost
2022-06-16T11:57:39.376+0000	dropping: parashar21Local.khona21Local
2022-06-16T11:57:42.374+0000	[####....................] parashar21Local.khona21Local	24.5MB/128MB (19.2%)
2022-06-16T11:57:45.377+0000	[##########..............] parashar21Local.khona21Local	54.1MB/128MB (42.3%)
2022-06-16T11:57:48.382+0000	[###############.........] parashar21Local.khona21Local	83.8MB/128MB (65.5%)
2022-06-16T11:57:51.386+0000	[#####################...] parashar21Local.khona21Local	113MB/128MB (88.3%)
2022-06-16T11:57:52.950+0000	[########################] parashar21Local.khona21Local	128MB/128MB (100.0%)
2022-06-16T11:57:52.950+0000	imported 39662 documents


In [7]:
kollection = db.khona21Local                   # Set the collection to work with
#kollection = db.khona21                   # Set the collection to work with
totalCharts = kollection.count_documents({})
dbSize = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
print(totalCharts, ' documents,',round(dbSize,2), " KB data")

39662  documents, 142321.85  KB data


#Report Generation Function
Please note that the generated report MS Office doc format **will be stored in the Colab VM drive** and needs to be downloaded to local machine for viewing

In [27]:
def genYogReport(yName, yDesc, ChartStyle,Sample = True):
    
    if Sample:
        df4Report = df.sample(5)
    else:
        df4Report = df
    
    selectedCharts = len(df)
    printCharts = len(df4Report)
    #printStatus = str(printCharts)+' charts of total '+str(totalCharts)+' printed.'
    
    printTxt0 = yName
    printTxt1 = yDesc
    printTxt2 = 'ref : https://www.astroyogi.com/kundli/yog'      # this is where the Yog definitions have been taken from 
    printTxt3 = str(selectedCharts)+' Charts selected from '+str(totalCharts)+' in database; '+str(printCharts)+' charts printed.'
    printTxt = printTxt0 + printTxt1 + '\n' + printTxt2 +'\n'+ printTxt3

    now = datetime.now(pytz.timezone('Asia/Kolkata'))
    fileName = 'p21_'+yName+'_'+ChartStyle+'_'+now.strftime("%H%M%S")+'.doc'
    curQueryStr = json.dumps(selCond)
    # ---------------------------------------------------------
    p21utilsR.R01_CreateReportDoc(curQueryStr, printTxt)                      # Creates MS Word Doc called p21.document
    # ----------------------------------------------------------
    for chart in range(len(df4Report)):
        chartData =df4Report.iloc[chart]
        #R51_processChart(chartData)
        p21utilsR.R511_parseChartData(chartData)              # Break up database data into useful pieces
                                            # Creates two primary dicts, GLon, GRet
        for ctype in ['Rashi','Navamsa']:
        #for ctype in ['Rashi']:
            p21.ChartType = ctype
            p21utils.R11_LocateGrahaInRashi()   # Converts Graha Long to Rashi positions
                                                # Rashi Num as well as Rashi name
            if ChartStyle == 'Bengal':
                p21utilsR.R12B_drawChart_Bengal()    # Draw Chart in Bengal style
            if ChartStyle == 'South':
                p21utilsR.R12B_drawChart_South()    # Draw Chart in South style
            if ChartStyle == 'North':
                p21utilsR.R12B_drawChart_North()    # Draw Chart in North style
            #
            
                                                
        #print(p21.GrahaBhava)
        #print(p21.LordBhav)
        p21utilsR.R512_FormatPage()
        #print('----')
    # ------------------------------------
    p21.document.save(fileName)
    


#Retrieve Charts 
Based on selective queries <br>
All output messages suppressed <br>
Reports will be available as DOC files in the VM drive <br>
Please specify style of report, 'Bengal', 'North' or 'South' while calling the report generation routine <br>
genSampleReport('Bengal',Sample = False), specifying Sample='False' will stop sampling and print reports for all charts retrieved. Otherwise, by default, only 5 charts, chosen at random will be printed in the DOC file <br>
genSampleReport() replaced by genYogReport() with two additional parameters, namely YogName and YogDecription that are printed in the report


## Budhaditya Yog

In [22]:
%%capture
yogName = 'Budhaditya'
yogDesc = ' -- formed by the conjunction of Sun and Mercury.  Considered to be a very auspicious Yoga.'            
selCond = {'GConjunctsG2.Su' : {'$in': ['Me']}}       # Su conjunct Me           
df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genYogReport(yogName, yogDesc,'Bengal')

## Gajakesari Yog

In [25]:
%%capture
yogName = 'GajaKesari_1'
yogDesc = ' -- When the moon and Jupiter is placed in the 1st, 4th, 7th and 10th house which forms the angular houses or Kendra'            
selCond = {"$and":
                    [
                        {'GConjunctsG2.Ju' : {'$in': ['Mo']}},      # Ju conjunct Mo 
                        { "$or" : [
                            {'GrahaBhava.Mo' : {'$eq': 1}},              # Mo in First House
                            {'GrahaBhava.Mo' : {'$eq': 4}},              # Mo in Fourth House
                            {'GrahaBhava.Mo' : {'$eq': 7}},              # Mo in Seventh House
                            {'GrahaBhava.Mo' : {'$eq': 10}}              # Mo in Tenth House
                          ]
                        }  

                    ]
}           
df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genYogReport(yogName, yogDesc,'Bengal')

In [26]:
%%capture
yogName = 'GajaKesari_2'
yogDesc = ' -- Jupiter is in Cancer, Venus in the ninth house and Jupiter in the seventh house of one's horoscope.'            
selCond = {"$and":
                    [
                        {'GConjunctsG2.Ju' : {'$in': ['Mo']}},      # Ju conjunct Mo 
                        { "$or" : [
                            {'GrahaBhava.Ju' : {'$eq': 1}},              # Mo in First House
                            {'GrahaBhava.Ju' : {'$eq': 4}},              # Mo in Fourth House
                            {'GrahaBhava.Ju' : {'$eq': 7}},              # Mo in Seventh House
                            {'GrahaBhava.Ju' : {'$eq': 10}}              # Mo in Tenth House
                          ]
                        }  

                    ]
}           
df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genYogReport(yogName, yogDesc,'Bengal')

##Raj Yog

In [30]:
%%capture
yogName = 'RajYoga'
yogDesc = ' -- Jupiter is in Cancer, Venus in the ninth house and Jupiter in the seventh house'   
selCond = {"$and":
                    [
                        {'GRashiN.Ju': {'$eq': 4}},               # Jupiter in Cancer
                        {'GrahaBhava.Ju' : {'$eq': 7}} ,            # Jupiter in 7th House
                        {'GrahaBhava.Ve' : {'$eq': 9}}             # Jupiter in 7th House

                    ]
}

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genYogReport(yogName, yogDesc,'Bengal')

#Old Templates
to be ignored

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}}       # Su conjunct Me 

                    ]
}

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}},      # Su conjunct Me 
                        {'GrahaBhava.Mo' : {'$eq': 1}}              # Mo in First House

                    ]
}

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}},      # Su conjunct Me 
                        {'GrahaBhava.Mo' : {'$eq': 1}} ,            # Mo in First House
                        {'LordBhav.4' : {'$eq': 5}}                 # 4th Lord in 5th House

                    ]
}

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal',Sample = False)

In [ ]:
!rm *.doc

In [ ]:
#df = pd.DataFrame(list(kollection.find({'pid.tags': {"$in": ['PostGraduate']}},{'pid.name':1,"_id":0,'GLon':1,'GRet':1})))
#df